# 1. Import Packages

In [276]:
import collections
import copy
import os
import pandas as pd
import tldextract

# 2. Read a csv file with URL analysis

## 2.1. Read a csv file

In [277]:
url_1stp_3rdp_df = pd.read_csv("First_Party_vs_Third_Party.csv")

In [278]:
url_ser = url_1stp_3rdp_df.loc[url_1stp_3rdp_df["first_party"].isna(), "url"]

In [4]:
url_ser

187                          https://amazon-presse.de/\n
229                            https://www.abebooks.de\n
231                             https://www.audible.de\n
232                             https://www.audible.de\n
233                    https://www.bookdepository.com/\n
234                              https://www.imdb.com/\n
236                 https://www.shopbop.com/de/welcome\n
238    https://www.zvab.com/index.do?ref=amazon&utm_m...
518                                 http://www.ebay.cn\n
536              http://global.gmarket.co.kr/Home/Main\n
554                       https://www.gittigidiyor.com\n
573    https://seal.digicert.com/seals/popup/?tag=BIE...
Name: url, dtype: object

## 2.2. url_l

In [11]:
url_l = []

for url in url_ser:
    url_ll = url.split("\n")
    url_l.append(url_ll[0])

url_l

['https://amazon-presse.de/',
 'https://www.abebooks.de',
 'https://www.audible.de',
 'https://www.audible.de',
 'https://www.bookdepository.com/',
 'https://www.imdb.com/',
 'https://www.shopbop.com/de/welcome',
 'https://www.zvab.com/index.do?ref=amazon&utm_medium=referral&utm_source=amazon.de',
 'http://www.ebay.cn',
 'http://global.gmarket.co.kr/Home/Main',
 'https://www.gittigidiyor.com',
 'https://seal.digicert.com/seals/popup/?tag=BIEBu4RK&url=www.ebay.de&lang=en&cbr=1637157663836']

## 2.3. Original hostnames

In [169]:
original_hostname_set = set(["amazon.de", "ebay.de"])

## 2.4. Matching (Could be prevented at the get go of the data injection)

In [170]:
original_hostname_l = len(url_l[:-4]) * ["amazon.de"]
original_hostname_l.extend(len(url_l[-4:]) * ["ebay.de"])
print(len(original_hostname_l) == len(url_l))
print(original_hostname_l)

True
['amazon.de', 'amazon.de', 'amazon.de', 'amazon.de', 'amazon.de', 'amazon.de', 'amazon.de', 'amazon.de', 'ebay.de', 'ebay.de', 'ebay.de', 'ebay.de']


## 2.5. original_url_df

In [172]:
original_url_df = pd.concat([pd.DataFrame(url_l), pd.DataFrame(original_hostname_l)], axis = 1)
original_url_df.columns = ["original_url", "original_hostname"]
original_url_df

,original_url,original_hostname
0,https://amazon-presse.de/,amazon.de
1,https://www.abebooks.de,amazon.de
2,https://www.audible.de,amazon.de
3,https://www.audible.de,amazon.de
4,https://www.bookdepository.com/,amazon.de
5,https://www.imdb.com/,amazon.de
6,https://www.shopbop.com/de/welcome,amazon.de
7,https://www.zvab.com/index.do?ref=amazon&utm_m...,amazon.de
8,http://www.ebay.cn,ebay.de
9,http://global.gmarket.co.kr/Home/Main,ebay.de


# 3. dig: (url to) hostname to ip

## 3.1. hostname_ip_d

In [107]:
hostname_ip_d = {}
hostname_l = []
# hostname_original_hostname_d = {}

for index, url in enumerate(url_l):
    extracted = tldextract.extract(url)
    hostname_s = "{}.{}".format(extracted.domain, extracted.suffix)
    hostname_l.append(hostname_s)
    # hostname_original_hostname_d[hostname_s] = original_hostname_l[index]
    
    cmd = "dig +short " + hostname_s
    ip_ll = os.popen(cmd).read().split("\n")
    
    for index, ip in enumerate(ip_ll):
        if index == 0:
            hostname_ip_d[hostname_s] = []
        if ip != "":
            hostname_ip_d[hostname_s].append(ip)
    
print(hostname_ip_d)
# print(len(hostname_ip_d) == len(hostname_original_hostname_d))
# print(hostname_original_hostname_d)
print()
print(hostname_l)

{'amazon-presse.de': ['52.208.0.37'], 'abebooks.de': ['13.226.237.121', '13.226.237.82', '13.226.237.115', '13.226.237.60'], 'audible.de': ['54.239.33.67', '52.95.118.67', '54.239.37.136'], 'bookdepository.com': ['54.239.39.203', '54.239.39.213', '54.239.39.193'], 'imdb.com': ['52.94.237.74', '52.94.228.167', '52.94.225.248'], 'shopbop.com': ['54.239.26.6', '54.239.25.228', '54.239.25.244'], 'zvab.com': ['13.226.228.66', '13.226.228.85', '13.226.228.96', '13.226.228.36'], 'ebay.cn': ['222.73.220.176'], 'gmarket.co.kr': ['183.111.134.78', '117.52.47.78'], 'gittigidiyor.com': ['93.187.67.10'], 'digicert.com': ['45.60.131.229', '45.60.121.229']}

['amazon-presse.de', 'abebooks.de', 'audible.de', 'audible.de', 'bookdepository.com', 'imdb.com', 'shopbop.com', 'zvab.com', 'ebay.cn', 'gmarket.co.kr', 'gittigidiyor.com', 'digicert.com']


## 3.2. original_hostname_to_ip_d

In [28]:
original_hostname_to_ip_d = {}

for hostname_s in original_hostname_set:
    
    cmd = "dig +short " + hostname_s
    ip_ll = os.popen(cmd).read().split("\n")
    
    for index, ip in enumerate(ip_ll):
        if index == 0:
            original_hostname_to_ip_d[hostname_s] = []
        if ip != "":
            original_hostname_to_ip_d[hostname_s].append(ip)
    
original_hostname_to_ip_d

{'ebay.de': ['64.4.253.29',
  '209.140.148.143',
  '216.113.181.253',
  '216.113.179.53',
  '66.211.175.229',
  '64.4.253.77',
  '66.211.172.37'],
 'amazon.de': ['54.239.39.102', '52.95.120.34', '176.32.108.185']}

## 3.3. initial df

In [91]:
hostname_l = []
ip_l = []

for hostname, ip_ll in hostname_ip_d.items():
    for ip in ip_ll:
        hostname_l.append(hostname)
        ip_l.append(ip)

print(hostname_l)
print(len(hostname_l) == len(ip_l))
print(ip_l)

['amazon-presse.de', 'amazon-presse.de', 'abebooks.de', 'abebooks.de', 'abebooks.de', 'abebooks.de', 'audible.de', 'audible.de', 'audible.de', 'bookdepository.com', 'bookdepository.com', 'bookdepository.com', 'imdb.com', 'imdb.com', 'imdb.com', 'shopbop.com', 'shopbop.com', 'shopbop.com', 'zvab.com', 'zvab.com', 'zvab.com', 'zvab.com', 'ebay.cn', 'gmarket.co.kr', 'gmarket.co.kr', 'gittigidiyor.com', 'digicert.com', 'digicert.com']
True
['54.72.217.61', '52.208.0.37', '108.139.1.80', '108.139.1.33', '108.139.1.16', '108.139.1.94', '52.95.118.67', '54.239.37.136', '54.239.33.67', '54.239.39.213', '54.239.39.203', '54.239.39.193', '52.94.225.248', '52.94.237.74', '52.94.228.167', '54.239.26.6', '54.239.25.244', '54.239.25.228', '13.227.65.104', '13.227.65.121', '13.227.65.74', '13.227.65.98', '222.73.220.176', '117.52.47.78', '183.111.134.78', '93.187.67.10', '45.60.121.229', '45.60.131.229']


In [92]:
df = pd.concat([pd.DataFrame(hostname_l), pd.DataFrame(ip_l)], axis=1)
df.columns = ['hostname','ip']
df

,hostname,ip
0,amazon-presse.de,54.72.217.61
1,amazon-presse.de,52.208.0.37
2,abebooks.de,108.139.1.80
3,abebooks.de,108.139.1.33
4,abebooks.de,108.139.1.16
5,abebooks.de,108.139.1.94
6,audible.de,52.95.118.67
7,audible.de,54.239.37.136
8,audible.de,54.239.33.67
9,bookdepository.com,54.239.39.213


# 4. whois: ip to AS Number and AS Name

## 4.1. whois

In [93]:
as_number_l = []
as_name_l = []

for ip in ip_l:
    cmd = f"whois -h whois.cymru.com {ip}"
    cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

    for index, item in enumerate(cymru_l):
        if index == 0:
            as_number_l.append(item.strip())
        if index == 2:
            as_name_l.append(item.strip())
            
print(as_number_l)
print(len(as_number_l) == len(as_name_l))
print(as_name_l)

['16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '16509', '4812', '3786', '4766', '39427', '19551', '19551']
True
['AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US', 'CHINANET-SH-AP China Telecom Group, CN', 'LGDACOM LG DACOM Corporation, KR', 'KIXS-AS-KR Korea Telecom, KR', 'GITTIGIDIYOR, TR', 'INCAPSULA, US', 'INCAPSULA, US']


## 4.2. df

In [94]:
colname_l = list(df.columns)
colname_l.append("as_number")
colname_l.append("as_name")

In [95]:
df = pd.concat([df, pd.DataFrame(as_number_l), pd.DataFrame(as_name_l)], axis=1)
df.columns = colname_l
df

,hostname,ip,as_number,as_name
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US"
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US"
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US"
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US"
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US"
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US"
6,audible.de,52.95.118.67,16509,"AMAZON-02, US"
7,audible.de,54.239.37.136,16509,"AMAZON-02, US"
8,audible.de,54.239.33.67,16509,"AMAZON-02, US"
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US"


## 4.3. original_hostname_to_as_number_d

In [139]:
original_as_number_l = []
original_as_name_l = []
original_hostname_to_as_number_d = copy.deepcopy(original_hostname_to_ip_d)
original_hostname_to_as_name_d = copy.deepcopy(original_hostname_to_ip_d)

for hostname, ip_ll in original_hostname_to_ip_d.items():
    original_hostname_to_as_number_d[hostname] = set()
    original_hostname_to_as_name_d[hostname] = set()
    for ip in ip_ll:
        cmd = f"whois -h whois.cymru.com {ip}"
        cymru_l = os.popen(cmd).read().split("\n")[1].split("|")

        for index, item in enumerate(cymru_l):
            if index == 0:
                original_as_number_l.append(item.strip())
                original_hostname_to_as_number_d[hostname].add(item.strip())
            if index == 2:
                original_as_name_l.append(item.strip())
                original_hostname_to_as_name_d[hostname].add(item.strip())
            
print(original_as_number_l)
print(len(original_as_number_l) == len(original_as_name_l))
print(original_as_name_l)
print(original_hostname_to_as_number_d)
print(original_hostname_to_as_name_d)

['11643', '11643', '11643', '11643', '11643', '11643', '11643', '16509', '16509', '16509']
True
['EBAY, US', 'EBAY, US', 'EBAY, US', 'EBAY, US', 'EBAY, US', 'EBAY, US', 'EBAY, US', 'AMAZON-02, US', 'AMAZON-02, US', 'AMAZON-02, US']
{'ebay.de': {'11643'}, 'amazon.de': {'16509'}}
{'ebay.de': {'EBAY, US'}, 'amazon.de': {'AMAZON-02, US'}}


## 4.4. original_hostname_original_as_number_df

In [178]:
k_l = []
v_l = []

for k, v_set in original_hostname_to_as_number_d.items():
    k_l.append(k)
    for v in v_set:
        v_l.append(v)
        
original_hostname_original_as_number_df = pd.concat([pd.DataFrame(k_l), pd.DataFrame(v_l)], axis=1)
del k_l, v_l
original_hostname_original_as_number_df.columns = ["original_hostname", "original_as_number"]
original_hostname_original_as_number_df

,original_hostname,original_as_number
0,ebay.de,11643
1,amazon.de,16509


## 4.5. original_df

In [190]:
original_df = pd.merge(original_url_df, original_hostname_original_as_number_df, how="left", on=["original_hostname"])
original_df

,original_url,original_hostname,original_as_number
0,https://amazon-presse.de/,amazon.de,16509
1,https://www.abebooks.de,amazon.de,16509
2,https://www.audible.de,amazon.de,16509
3,https://www.audible.de,amazon.de,16509
4,https://www.bookdepository.com/,amazon.de,16509
5,https://www.imdb.com/,amazon.de,16509
6,https://www.shopbop.com/de/welcome,amazon.de,16509
7,https://www.zvab.com/index.do?ref=amazon&utm_m...,amazon.de,16509
8,http://www.ebay.cn,ebay.de,11643
9,http://global.gmarket.co.kr/Home/Main,ebay.de,11643


## 4.6. drop_duplicates()

In [191]:
original_df = original_df.drop_duplicates().reset_index(drop=True)
original_df

,original_url,original_hostname,original_as_number
0,https://amazon-presse.de/,amazon.de,16509
1,https://www.abebooks.de,amazon.de,16509
2,https://www.audible.de,amazon.de,16509
3,https://www.bookdepository.com/,amazon.de,16509
4,https://www.imdb.com/,amazon.de,16509
5,https://www.shopbop.com/de/welcome,amazon.de,16509
6,https://www.zvab.com/index.do?ref=amazon&utm_m...,amazon.de,16509
7,http://www.ebay.cn,ebay.de,11643
8,http://global.gmarket.co.kr/Home/Main,ebay.de,11643
9,https://www.gittigidiyor.com,ebay.de,11643


# 5. AS Number to AS2Org_OrgID

In [97]:
# asndb = pyasn.pyasn('./ipasn.dat')
asn_org_id = {}
org_id_name = {}

with open('./20220101.as-org2info.txt', 'r') as caida_file:
    for line in caida_file.readlines():
        if not line.startswith("#"):
            items = line.strip().split("|")
            if len(items) == 5:
                org_id_name[items[0]] = items[2]
            elif len(items) == 6:
                asn_org_id[int(items[0])] = items[3]
            else:
                assert False

In [98]:
asn_org_id

{1: 'LPL-141-ARIN',
 2: 'UNIVER-19-ARIN',
 3: 'MIT-2-ARIN',
 4: 'USC-32-ARIN',
 5: 'WGL-117-ARIN',
 6: 'ATOS-Z-ARIN',
 7: 'ORG-TDSA4-RIPE',
 8: 'RICEUN-ARIN',
 9: 'CARNEG-Z-ARIN',
 10: 'CCICC-ARIN',
 11: 'HARVAR-ARIN',
 12: 'NYU-ARIN',
 13: 'HEADQU-3-ARIN',
 14: 'COLUMB-ARIN',
 15: 'DYNAMI-109-ARIN',
 16: 'LBNL-ARIN',
 17: 'PURDUE-ARIN',
 18: 'UTAA-ARIN',
 19: 'LEIDO-ARIN',
 20: 'UNIVER-40-Z-ARIN',
 21: 'THERAN-Z-ARIN',
 22: 'NNICN-1-ARIN',
 23: 'NASA-ARIN',
 24: 'NASA-ARIN',
 25: 'UCAB-ARIN',
 26: 'CORNEL-ARIN',
 27: 'UNIVER-262-Z-ARIN',
 28: 'ORG-DZFL1-RIPE',
 29: 'YALEUN-ARIN',
 30: 'SRIINT-11-ARIN',
 31: 'CIT-4-ARIN',
 32: 'STANFO-ARIN',
 33: 'HP-ARIN',
 34: 'UNIVER-19-ARIN',
 35: 'THEMIT-1-Z-ARIN',
 36: 'SRIINT-ARIN',
 37: 'NNICN-1-ARIN',
 38: 'UIUC-ARIN',
 39: 'DNIC-ARIN',
 40: 'MIT-2-ARIN',
 41: 'NASA-ARIN',
 42: 'WOODYN-ARIN',
 43: 'BNL-ARIN',
 44: 'LLNL-1-ARIN',
 45: 'LLNL-1-ARIN',
 46: 'RUTGER-ARIN',
 47: 'USC-6-ARIN',
 48: 'NNICN-1-ARIN',
 49: 'NBS-46-ARIN',
 50: 'ORNL-ARIN'

In [99]:
org_id_name

{'111S-ARIN': 'One Eleven Internet Services',
 '1800CO-2-ARIN': '1-800 Contacts, Inc.',
 '1800FL-ARIN': '1-800-Flowers.com, Inc.',
 '1800H-ARIN': '1-800-HOSTING, Inc.',
 '1881CS-ARIN': '1881CS/XPC',
 '1FBU-ARIN': '1st Financial Bank USA',
 '1GPUC-ARIN': '1798 Global Partners (USA), Corp.',
 '1STP-ARIN': 'FIRST STEP INTERNET, LLC',
 '1UWEB-ARIN': '1U Web, INC.',
 '280INC-ARIN': '280, Inc.',
 '2CHEC-ARIN': '2checkout.com',
 '2CI-3-ARIN': '21st Century Insurance Group',
 '2CO-ARIN': '21ST CENTURY ONCOLOGY, INC.',
 '2HF-1-ARIN': '24 Hour Fitness USA, Inc.',
 '2KGAM-ARIN': '2K GAMES, Inc.',
 '2SS-3-ARIN': '2nd Story Software, Inc.',
 '2WIRE-1-ARIN': '2Wire Inc.',
 '3CL-3-ARIN': '360 Communications, LLC',
 '3ECOMP-ARIN': '3E COMPANY ENVIRONMENTAL, ECOLOGICAL AND ENGINEERING',
 '3FORM-ARIN': '3form Inc.',
 '3IN-ARIN': '3Men@Work Integrated Networks, Inc.',
 '3MCOMP-Z-ARIN': '3M Company',
 '3UTEL-ARIN': 'Telecom North America Inc.',
 '3ZCAN-ARIN': '1538556 Ontario Inc.',
 '3ZNET-ARIN': '3z.net

In [100]:
org_id_l = []
for as_numbrer in as_number_l:
    org_id_l.append(asn_org_id[int(as_numbrer)])

In [101]:
colname_l = list(df.columns)
colname_l.append("org_id")

In [102]:
df = pd.concat([df, pd.DataFrame(org_id_l)], axis=1)
df.columns = colname_l
df

,hostname,ip,as_number,as_name,org_id
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US",AMAZON-4-ARIN
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US",AMAZON-4-ARIN
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US",AMAZON-4-ARIN
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US",AMAZON-4-ARIN
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US",AMAZON-4-ARIN
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US",AMAZON-4-ARIN
6,audible.de,52.95.118.67,16509,"AMAZON-02, US",AMAZON-4-ARIN
7,audible.de,54.239.37.136,16509,"AMAZON-02, US",AMAZON-4-ARIN
8,audible.de,54.239.33.67,16509,"AMAZON-02, US",AMAZON-4-ARIN
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US",AMAZON-4-ARIN


# 6. Columns: is_first_party and matching_variable

In [103]:
is_first_party_l = df.shape[0] * [False]

In [104]:
matching_variable_l = df.shape[0] * [None]

In [105]:
colname_l = list(df.columns)
colname_l.append("is_first_party")
colname_l.append("matching_variable")
df = pd.concat([df, pd.DataFrame(is_first_party_l), pd.DataFrame(matching_variable_l)], axis=1)
df.columns = colname_l
df

,hostname,ip,as_number,as_name,org_id,is_first_party,matching_variable
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
6,audible.de,52.95.118.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
7,audible.de,54.239.37.136,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
8,audible.de,54.239.33.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None


# 7. Matching hostname and Original Hostname

## 7.1. hostnames_df

In [195]:
hostnames_df = pd.concat([pd.DataFrame(hostname_l), pd.DataFrame(original_hostname_l)], axis=1)
hostnames_df.columns = ["hostname", "original_hostname"]
hostnames_df

,hostname,original_hostname
0,amazon-presse.de,amazon.de
1,abebooks.de,amazon.de
2,audible.de,amazon.de
3,audible.de,amazon.de
4,bookdepository.com,amazon.de
5,imdb.com,amazon.de
6,shopbop.com,amazon.de
7,zvab.com,amazon.de
8,ebay.cn,ebay.de
9,gmarket.co.kr,ebay.de


In [197]:
hostnames_df = hostnames_df.drop_duplicates().reset_index(drop=True)
hostnames_df

,hostname,original_hostname
0,amazon-presse.de,amazon.de
1,abebooks.de,amazon.de
2,audible.de,amazon.de
3,bookdepository.com,amazon.de
4,imdb.com,amazon.de
5,shopbop.com,amazon.de
6,zvab.com,amazon.de
7,ebay.cn,ebay.de
8,gmarket.co.kr,ebay.de
9,gittigidiyor.com,ebay.de


## 7.2. concat([hostnames_df, original_df])

In [212]:
super_original_df = pd.concat([pd.DataFrame(hostnames_df["hostname"]), original_df], axis=1)
super_original_df

,hostname,original_url,original_hostname,original_as_number
0,amazon-presse.de,https://amazon-presse.de/,amazon.de,16509
1,abebooks.de,https://www.abebooks.de,amazon.de,16509
2,audible.de,https://www.audible.de,amazon.de,16509
3,bookdepository.com,https://www.bookdepository.com/,amazon.de,16509
4,imdb.com,https://www.imdb.com/,amazon.de,16509
5,shopbop.com,https://www.shopbop.com/de/welcome,amazon.de,16509
6,zvab.com,https://www.zvab.com/index.do?ref=amazon&utm_m...,amazon.de,16509
7,ebay.cn,http://www.ebay.cn,ebay.de,11643
8,gmarket.co.kr,http://global.gmarket.co.kr/Home/Main,ebay.de,11643
9,gittigidiyor.com,https://www.gittigidiyor.com,ebay.de,11643


## 7.3. df LEFT JOIN original_df

In [214]:
without_original_df = copy.deepcopy(df)

In [215]:
df = pd.merge(df, super_original_df, how="left", on=["hostname"])
print(df.shape)

(28, 10)


In [216]:
df

,hostname,ip,as_number,as_name,org_id,is_first_party,matching_variable,original_url,original_hostname,original_as_number
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://amazon-presse.de/,amazon.de,16509
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://amazon-presse.de/,amazon.de,16509
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.abebooks.de,amazon.de,16509
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.abebooks.de,amazon.de,16509
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.abebooks.de,amazon.de,16509
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.abebooks.de,amazon.de,16509
6,audible.de,52.95.118.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.audible.de,amazon.de,16509
7,audible.de,54.239.37.136,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.audible.de,amazon.de,16509
8,audible.de,54.239.33.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.audible.de,amazon.de,16509
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,https://www.bookdepository.com/,amazon.de,16509


## 7.4. Column rearrangement

In [218]:
colname10_l = df.columns.to_list()
colname10_l

['hostname',
 'ip',
 'as_number',
 'as_name',
 'org_id',
 'is_first_party',
 'matching_variable',
 'original_url',
 'original_hostname',
 'original_as_number']

In [220]:
new_colname10_l = colname10_l[:-3] + [colname10_l[-1]] + [colname10_l[-2]] + [colname10_l[-3]]

In [221]:
df = df[new_colname10_l]
print(df.shape)
df

(28, 10)


,hostname,ip,as_number,as_name,org_id,is_first_party,matching_variable,original_as_number,original_hostname,original_url
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://amazon-presse.de/
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://amazon-presse.de/
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.abebooks.de
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.abebooks.de
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.abebooks.de
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.abebooks.de
6,audible.de,52.95.118.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.audible.de
7,audible.de,54.239.37.136,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.audible.de
8,audible.de,54.239.33.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.audible.de
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US",AMAZON-4-ARIN,False,None,16509,amazon.de,https://www.bookdepository.com/


# 8. matching_variable

## 8.1. Base on as_number

In [248]:
before_if_df = copy.deepcopy(df)

In [249]:
df.loc[df["as_number"] == df["original_as_number"], "is_first_party"] = True
df.loc[df["as_number"] == df["original_as_number"], "matching_variable"] = "3"

In [251]:
print(df.shape)
df

(28, 10)


,hostname,ip,as_number,as_name,org_id,is_first_party,matching_variable,original_as_number,original_hostname,original_url
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://amazon-presse.de/
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://amazon-presse.de/
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.abebooks.de
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.abebooks.de
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.abebooks.de
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.abebooks.de
6,audible.de,52.95.118.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.audible.de
7,audible.de,54.239.37.136,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.audible.de
8,audible.de,54.239.33.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.audible.de
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,3,16509,amazon.de,https://www.bookdepository.com/


## 8.2. Based on as_name

In [232]:
target_company_s = set(["amazon", "ebay", "facebook"])

In [252]:
before_adding_lower_columns_df = copy.deepcopy(df)

In [253]:
df["lower_as_name"] = df["as_name"].str.lower()

In [254]:
for target_company in target_company_s:
    df.loc[df["lower_as_name"].str.contains(target_company), "is_first_party"] = True
    df.loc[df["lower_as_name"].str.contains(target_company), "matching_variable"] += "4"

## 8.3. Based on org_id

In [255]:
df["lower_org_id"] = df["org_id"].str.lower()

In [256]:
for target_company in target_company_s:
    df.loc[df["lower_org_id"].str.contains(target_company), "is_first_party"] = True
    df.loc[df["lower_org_id"].str.contains(target_company), "matching_variable"] += "5"

In [258]:
df = df[new_colname10_l]

In [259]:
print(df.shape)
df

(28, 10)


,hostname,ip,as_number,as_name,org_id,is_first_party,matching_variable,original_as_number,original_hostname,original_url
0,amazon-presse.de,54.72.217.61,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://amazon-presse.de/
1,amazon-presse.de,52.208.0.37,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://amazon-presse.de/
2,abebooks.de,108.139.1.80,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.abebooks.de
3,abebooks.de,108.139.1.33,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.abebooks.de
4,abebooks.de,108.139.1.16,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.abebooks.de
5,abebooks.de,108.139.1.94,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.abebooks.de
6,audible.de,52.95.118.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.audible.de
7,audible.de,54.239.37.136,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.audible.de
8,audible.de,54.239.33.67,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.audible.de
9,bookdepository.com,54.239.39.213,16509,"AMAZON-02, US",AMAZON-4-ARIN,True,345,16509,amazon.de,https://www.bookdepository.com/


# 9. Manual matching

In [262]:
df.tail(6)

,hostname,ip,as_number,as_name,org_id,is_first_party,matching_variable,original_as_number,original_hostname,original_url
22,ebay.cn,222.73.220.176,4812,"CHINANET-SH-AP China Telecom Group, CN",ORG-CT1-AP-APNIC,False,None,11643,ebay.de,http://www.ebay.cn
23,gmarket.co.kr,117.52.47.78,3786,"LGDACOM LG DACOM Corporation, KR",@aut-9688-APNIC,False,None,11643,ebay.de,http://global.gmarket.co.kr/Home/Main
24,gmarket.co.kr,183.111.134.78,4766,"KIXS-AS-KR Korea Telecom, KR",@family-61705,False,None,11643,ebay.de,http://global.gmarket.co.kr/Home/Main
25,gittigidiyor.com,93.187.67.10,39427,"GITTIGIDIYOR, TR",ORG-GBTS1-RIPE,False,None,11643,ebay.de,https://www.gittigidiyor.com
26,digicert.com,45.60.121.229,19551,"INCAPSULA, US",INCAP-5-ARIN,False,None,11643,ebay.de,https://seal.digicert.com/seals/popup/?tag=BIE...
27,digicert.com,45.60.131.229,19551,"INCAPSULA, US",INCAP-5-ARIN,False,None,11643,ebay.de,https://seal.digicert.com/seals/popup/?tag=BIE...


## 9.1. ebay.cn

In [ ]:
hostname = "ebay.cn"

In [263]:
df.loc[df["hostname"] == hostname, "is_first_party"] = True
df.loc[df["hostname"] == hostname, "matching_variable"] = "1"

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 9.2. annotation_url

In [265]:
df["annotation_url"] = None

/var/folders/zt/v6h22_cj7dv_pbkt59s8mxbh0000gn/T/ipykernel_1967/1800894107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["annotation_url"] = None


## 9.3. gmarket.co.kr

In [268]:
hostname = "gmarket.co.kr"

In [269]:
df.loc[df["hostname"] == hostname, "is_first_party"] = True
df.loc[df["hostname"] == hostname, "annotation_url"] = "https://www.ebayinc.com/stories/news/breaking-ebay-expands-in-asia-gmarket-purchase-announced/"

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 9.4. gittigidiyor.com

In [271]:
hostname = "gittigidiyor.com"

In [272]:
df.loc[df["hostname"] == hostname, "is_first_party"] = True
df.loc[df["hostname"] == hostname, "annotation_url"] = "gittigidiyor.com"

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## 9.5. Save as CSV

In [275]:
df.to_csv("First_Party_vs_Third_Party_11_columns.csv")